# import necessary libraries

In [ ]:
import argparse
import tensorflow as tf
import numpy as np
import cv2
from utils import check_tf_version, load_labels, draw_detections

# Set numpy print precision for clean output (optional)

In [ ]:
np.set_printoptions(precision=2)

# -------------------------------------------------------
# Load the TensorFlow Lite model and allocate tensors
# -------------------------------------------------------

In [ ]:
interpreter = tf.lite.Interpreter(model_path="coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/detect.tflite")
interpreter.allocate_tensors()

# -------------------------------------------------------
# Get input and output details of the model
# -------------------------------------------------------

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get input shape (height & width expected by model)
input_height = input_details[0]['shape'][1]
input_width = input_details[0]['shape'][2]


# Check output layer name to determine if this model was created with TF2 or TF1,
# because outputs are ordered differently for TF2 and TF1 models

In [ ]:
boxes_idx, classes_idx, scores_idx = check_tf_version(output_details)      

# -------------------------------------------------------
# Load and preprocess the image
# -------------------------------------------------------

In [ ]:
image = cv2.imread(args.image)  # Load image
if image is None:
    raise FileNotFoundError(f"Image not found at {args.image}")
image_resized = cv2.resize(image, (input_width, input_height))  # Resize for model input
input_data = np.expand_dims(image_resized, axis=0).astype(np.uint8) # Add batch dimension and convert to uint8

# Get image shape for drawing boxes later
image_height, image_width, _ = image.shape

# -------------------------------------------------------
# Load labels (class names)
# -------------------------------------------------------

In [ ]:
labels = load_labels("coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/labelmap.txt")  # Load labels from labelmap file

# -------------------------------------------------------
# Run inference (object detection)
# -------------------------------------------------------

In [ ]:
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Retrieve detection results

In [ ]:
boxes = interpreter.get_tensor(output_details[boxes_idx]['index'])[0]    # Bounding boxes
classes = interpreter.get_tensor(output_details[classes_idx]['index'])[0]  # Class IDs
scores = interpreter.get_tensor(output_details[scores_idx]['index'])[0]    # Confidence scores

# -------------------------------------------------------
# Process detections and draw bounding boxes
# -------------------------------------------------------

In [ ]:
detections = draw_detections(image, boxes, classes, scores, labels, args.threshold)

# -------------------------------------------------------
# Display and save the output image
# -------------------------------------------------------

In [ ]:
cv2.imshow('Object Detection', image)
cv2.imwrite(args.output, image)
cv2.waitKey(0)
cv2.destroyAllWindows()
